In [3]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random

In [1]:
!ls

bar_chart.html	draft		     PlotLine.ipynb  v1_dict.json
BarChart.ipynb	googleplaystore.csv  SP500.csv


In [4]:
df = pd.read_csv("SP500.csv")
df.head()

,Date,Value
0,1977-09-02,97.45
1,1977-09-09,96.37
2,1977-09-16,96.48
3,1977-09-23,95.04
4,1977-09-30,96.53


In [5]:
sp500_dict = df.to_dict(orient="records")
sp500_dict[:5]

[{'Date': '1977-09-02', 'Value': 97.45},
 {'Date': '1977-09-09', 'Value': 96.37},
 {'Date': '1977-09-16', 'Value': 96.48},
 {'Date': '1977-09-23', 'Value': 95.04},
 {'Date': '1977-09-30', 'Value': 96.53}]

In [18]:
js_text_template = Template('''
// set the dimensions and margins of the graph
var margin = {top: 10, right: 30, bottom: 30, left: 60},
    width = 460 - margin.left - margin.right,
    height = 400 - margin.top - margin.bottom;

var y = d3.scaleLinear().range([height,0]);
var x = d3.scaleBand().range([0, width]).padding(0.2);
//d3.scale.linear().range([0,width]);

// append the svg object to the body of the page
var svg = d3.select("#$maid")
  .append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");
          
svg.append("text")
.attr("transform","translate(100,0)")
.attr("x",50)
.attr("y",10)
.attr("class","title")
.text("SP500 plot line chart");

var data = $python_data;
data.forEach(function(d) {
    d.Date = d3.timeParse("%Y-%m-%d")(d.Date);
    d.Value = +d.Value;
  });
console.log(data);
var x = d3.scaleTime()
      .domain(d3.extent(data, function(d) { return d.Date; }))
      .range([ 0, width ]);
    svg.append("g")
      .attr("transform", "translate(0," + height + ")")
      .call(d3.axisBottom(x));

// Add Y axis
var y = d3.scaleLinear()
      .domain([0, d3.max(data, function(d) { return +d.Value; })])
      .range([ height, 0 ]);
    svg.append("g")
      .call(d3.axisLeft(y));

// Add the line
svg.append("path")
      .datum(data)
      .attr("fill", "none")
      .attr("stroke", "steelblue")
      .attr("stroke-width", 1.5)
      .attr("d", d3.line()
        .x(function(d) { 
        // console.log(d.Date);
        return x(d.Date); })
        .y(function(d) { return y(d.Value) })
        );

''')

In [19]:
html_template = Template('''
<script src='https://d3js.org/d3.v4.js'></script>
    <style>
    .dot{
        stroke: #000;
    }
    .legend{
        color:blue;
    }
    .bar {
       fill: green;
     }
    
    .axis path,
    .axis line{
        fill: none;
        shape-rendering:crispEdges;
    
    }
    </style>
    <div id="$maid"></div>
    <script>$js_text</script>
''')
chart_id = "plotline-id"
js_text = js_text_template.substitute({
    "python_data":json.dumps(sp500_dict),
    "maid":chart_id
})

HTML(html_template.substitute({"maid":chart_id,"js_text":js_text}))